# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [2]:
%help

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.5 



# Available Magic Commands

## Sessions Magic

----
    %help                             Return a list of descriptions and input types for all magic commands. 
    %profile            String        Specify a profile in your aws configuration to use as the credentials provider.
    %region             String        Specify the AWS region in which to initialize a session. 
                                      Default from ~/.aws/config on Linux or macOS, 
                                      or C:\Users\ USERNAME \.aws\config" on Windows.
    %idle_timeout       Int           The number of minutes of inactivity after which a session will timeout. 
                                      Default: 2880 minutes (48 hours).
    %timeout            Int           The number of minutes after which a session will timeout. 
                                      Default: 2880 minutes (48 hours).
    %session_id_prefix  String        Define a String that will precede all session IDs in the format 
                                      [session_id_prefix]-[session_id]. If a session ID is not provided,
                                      a random UUID will be generated.
    %status                           Returns the status of the current Glue session including its duration, 
                                      configuration and executing user / role.
    %session_id                       Returns the session ID for the running session.
    %list_sessions                    Lists all currently running sessions by ID.
    %stop_session                     Stops the current session.
    %glue_version       String        The version of Glue to be used by this session. 
                                      Currently, the only valid options are 2.0, 3.0 and 4.0. 
                                      Default: 2.0.
    %reconnect          String        Specify a live session ID to switch/reconnect to the sessions.
----

## Selecting Session Types

----
    %streaming          String        Sets the session type to Glue Streaming.
    %etl                String        Sets the session type to Glue ETL.
    %glue_ray           String        Sets the session type to Glue Ray.
    %session_type       String        Specify a session_type to be used. Supported values: streaming, etl and glue_ray. 
----

## Glue Config Magic 
*(common across all session types)*

----

    %%configure         Dictionary    A json-formatted dictionary consisting of all configuration parameters for 
                                      a session. Each parameter can be specified here or through individual magics.
    %iam_role           String        Specify an IAM role ARN to execute your session with.
                                      Default from ~/.aws/config on Linux or macOS, 
                                      or C:\Users\%USERNAME%\.aws\config` on Windows.
    %number_of_workers  int           The number of workers of a defined worker_type that are allocated 
                                      when a session runs.
                                      Default: 5.
    %additional_python_modules  List  Comma separated list of additional Python modules to include in your cluster 
                                      (can be from Pypi or S3).
    %%tags        Dictionary          Specify a json-formatted dictionary consisting of tags to use in the session.
    
    %%assume_role Dictionary, String  Specify a json-formatted dictionary or an IAM role ARN string to create a session 
                                      for cross account access.
                                      E.g. {valid arn}
                                      %%assume_role 
                                      'arn:aws:iam::XXXXXXXXXXXX:role/AWSGlueServiceRole' 
                                      E.g. {credentials}
                                      %%assume_role
                                      {
                                            "aws_access_key_id" : "XXXXXXXXXXXX",
                                            "aws_secret_access_key" : "XXXXXXXXXXXX",
                                            "aws_session_token" : "XXXXXXXXXXXX"
                                       }
----

                                      
## Magic for Spark Sessions (ETL & Streaming)

----
    %worker_type        String        Set the type of instances the session will use as workers. 
    %connections        List          Specify a comma separated list of connections to use in the session.
    %extra_py_files     List          Comma separated list of additional Python files From S3.
    %extra_jars         List          Comma separated list of additional Jars to include in the cluster.
    %spark_conf         String        Specify custom spark configurations for your session. 
                                      E.g. %spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer
----
                                      
## Magic for Ray Session

----
    %min_workers        Int           The minimum number of workers that are allocated to a Ray session. 
                                      Default: 1.
    %object_memory_head Int           The percentage of free memory on the instance head node after a warm start. 
                                      Minimum: 0. Maximum: 100.
    %object_memory_worker Int         The percentage of free memory on the instance worker nodes after a warm start. 
                                      Minimum: 0. Maximum: 100.
----

## Action Magic

----

    %%sql               String        Run SQL code. All lines after the initial %%sql magic will be passed
                                      as part of the SQL code.  
    %matplot      Matplotlib figure   Visualize your data using the matplotlib library.
                                      E.g. 
                                      import matplotlib.pyplot as plt
                                      # Set X-axis and Y-axis values
                                      x = [5, 2, 8, 4, 9]
                                      y = [10, 4, 8, 5, 2]
                                      # Create a bar chart 
                                      plt.bar(x, y) 
                                      # Show the plot
                                      %matplot plt    
    %plotly            Plotly figure  Visualize your data using the plotly library.
                                      E.g.
                                      import plotly.express as px
                                      #Create a graphical figure
                                      fig = px.line(x=["a","b","c"], y=[1,3,2], title="sample figure")
                                      #Show the figure
                                      %plotly fig

  
                
----



####  Run this cell to set up and start your interactive session.


In [1]:
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

import sys
import json
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame
from pyspark.sql import functions as F
from pyspark.sql.functions import explode
from pyspark.sql.types import StructType, ArrayType
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.5 
Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 4.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Idle Timeout: 2880
Session ID: 1238c3ab-279d-4d75-bb06-560cbfc1799d
Applying the following default arguments:
--glue_kernel_version 1.0.5
--enable-glue-datacatalog true
Waiting for session 1238c3ab-279d-4d75-bb06-560cbfc1799d to get into ready status...
Session 1238c3ab-279d-4d75-bb06-560cbfc1799d ha

# Definimos la función que necesitaremos usar para formatear correctamente los archivos raw.

Ya que el método explode solo funciona con elementos tipo Array, debemos crear una función que aplane elementos tipo Struct.

* La función recibirá el schema del dataframe, iteraremos cada campo y mediante recursión iremos desanidando todos aquellos campos de tipo estructura.


In [2]:
# Definimos la función para aplanar las StructTypes que encontremos en el DataFrame
def flatten(schema, prefix=None):
    fields=[]
    for field in schema.fields:
        name=f"{prefix}.{field.name}" if prefix else f"{field.name}"
        dtype=field.dataType
        if isinstance(dtype, StructType):
            fields+=flatten(dtype, prefix=name)
        else:
            fields.append(name)
    return fields

# El plan a seguir es:

* Cargar los archivos con los que vamos a trabajar. 
* Aplanar los dataframes. 
* Unir los datos.

In [3]:
bucket_name="s3://bucket-for-requests/data/api_database/requests_json/"
input_files=["housing_data.json", "housing_data_2.json", "housing_data_3.json"]

In [4]:
dataframes=[]
for input_file in input_files:
    input_path=f"{bucket_name}{input_file}"
    df=spark.read.json(input_path)
    dataframes.append(df)

In [13]:
explode_dataframes=[]
for dataframe in dataframes:
    dataframe=dataframe.select(explode("elementList").alias("flattened_elementList"))
    explode_dataframes.append(dataframe)

In [14]:
flattened_dataframes=[]
for dataframe in explode_dataframes:
    flattened_df=dataframe.select(flatten(dataframe.schema))
    flattened_dataframes.append(flattened_df)

flattened_df1,flattened_df2,flattened_df3=flattened_dataframes

In [18]:
count=1
for df in flattened_dataframes: # Realizamos el conteo de filas para chequear la correcta implementación de los union. En total deberían ser 7450 filas
    rows=df.count()
    print(f"El dataframe {count} tiene {rows} filas0")
    count+=1

El dataframe 1 tiene 2500 filas
El dataframe 2 tiene 2950 filas
El dataframe 3 tiene 2000 filas


 # Realizamos las operaciones union pertinentes para trabajar con un único dataframe.

In [30]:
first_union_df=flattened_df1.union(flattened_df2)
print(f"El dataframe contiene {first_union_df.count()} filas")
last_union_df=first_union_df.union(flattened_df3)
print(f"El dataframe contiene {last_union_df.count()} filas") # Observamos que el conteo de filas es correcto

El dataframe contiene 5450 filas
El dataframe contiene 7450 filas


In [33]:
last_union_df.printSchema()# Observamos la estructura final del archivo

root
 |-- address: string (nullable = true)
 |-- bathrooms: long (nullable = true)
 |-- country: string (nullable = true)
 |-- description: string (nullable = true)
 |-- subTypology: string (nullable = true)
 |-- typology: string (nullable = true)
 |-- distance: string (nullable = true)
 |-- district: string (nullable = true)
 |-- exterior: boolean (nullable = true)
 |-- externalReference: string (nullable = true)
 |-- floor: string (nullable = true)
 |-- has360: boolean (nullable = true)
 |-- has3DTour: boolean (nullable = true)
 |-- hasLift: boolean (nullable = true)
 |-- hasPlan: boolean (nullable = true)
 |-- hasStaging: boolean (nullable = true)
 |-- hasVideo: boolean (nullable = true)
 |-- groupDescription: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- municipality: string (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- newDevelopment: boolean (nullable = true)
 |-- newDevelopmentFinished: boolean

In [36]:
dyf_flattened=DynamicFrame.fromDF(last_union_df, glueContext, "dyf_flattened") # Convertimos el DataFrame a un DynamicFrame
dyf_flattened.printSchema()
dyf_flattened.show(2)

root
|-- address: string
|-- bathrooms: long
|-- country: string
|-- description: string
|-- subTypology: string
|-- typology: string
|-- distance: string
|-- district: string
|-- exterior: boolean
|-- externalReference: string
|-- floor: string
|-- has360: boolean
|-- has3DTour: boolean
|-- hasLift: boolean
|-- hasPlan: boolean
|-- hasStaging: boolean
|-- hasVideo: boolean
|-- groupDescription: string
|-- latitude: double
|-- longitude: double
|-- municipality: string
|-- neighborhood: string
|-- newDevelopment: boolean
|-- newDevelopmentFinished: boolean
|-- numPhotos: long
|-- operation: string
|-- hasParkingSpace: boolean
|-- isParkingSpaceIncludedInPrice: boolean
|-- parkingSpacePrice: double
|-- price: double
|-- priceByArea: double
|-- amount: double
|-- currencySuffix: string
|-- formerPrice: double
|-- priceDropPercentage: long
|-- priceDropValue: long
|-- propertyCode: string
|-- propertyType: string
|-- province: string
|-- rooms: long
|-- showAddress: boolean
|-- size: doub

# Terminamos en este apartado, guardando el archivo correctamente formateado pero RAW.

In [67]:
# Con el dynamic frame ahora podemos escribir el archivo en formato parquet para construir el delta lake.
glueContext.write_dynamic_frame.from_options(frame=dyf_flattened, connection_type="s3", connection_options={"path": "s3://bucket-for-requests/data/api_database/bronze/"}, format="parquet")

In [ ]:
job.commit()